# 🧠 Day 3 – SQL via Python: NYC School Data Exploration
In this notebook, you'll connect to a PostgreSQL database and execute SQL queries to explore NYC school data.

## 🔌 Step 1: Import Libraries

In [32]:
import pandas as pd
import psycopg2

#Test
print(pd.__version__)
print(psycopg2.__version__)

2.2.3
2.9.10 (dt dec pq3 ext lo64)


## 🔐 Step 2: Connect to the Database

In [33]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

## 🔍 Step 3: Run a Test Query

In [34]:
# Code to check table high_school_directory
query = "SELECT * FROM nyc_schools.high_school_directory;"
df = pd.read_sql(query, conn)
df.head()


C:\Users\levon\AppData\Local\Temp\ipykernel_11180\1174011080.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,start_time,end_time,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,location,phone_number2,school_email,website,subway,bus,grades2018,finalgrades,total_students,extracurricular_activities,school_sports,attendance_rate,pct_stu_enough_variety,pct_stu_safe,school_accessibility_description,directions1,requirement1,requirement2,requirement3,requirement4,requirement5,program1,code1,interest1,method1,seats9ge1,grade9gefilledflag1,grade9geapplicants1,seats9swd1,grade9swdfilledflag1,grade9swdapplicants1,campus_name,building_borough,building_location,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,zip_codes,community_districts,borough_boundaries,city_council_districts,police_precincts,primary_address_line_1,city,state_code,postcode,school_type,overview_paragraph,program_highlights,language_classes,advancedplacement_courses,online_ap_courses,online_language_courses,psal_sports_boys,psal_sports_girls,psal_sports_coed,partner_cbo,partner_hospital,partner_highered,partner_cultural,partner_nonprofit,partner_corporate,partner_financial,partner_other,addtl_info1,addtl_info2,se_services,ell_programs,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,7:45 AM,2:05 PM,Priority to Queens students or residents who a...,Then to New York City residents who attend an ...,Then to Queens students or residents,Then to New York City residents,,,,,,,None,None,None,http://schools.nyc.gov/schoolportals/27/Q260,A to Beach 25th St-Wavecrest,"Q113, Q22",None,None,412.0,"After-school Program, Book, Writing, Homework ...","Step Team, Modern Dance, Hip Hop Dance",None,None,None,Not Functionally Accessible,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Far Rockaway Educational Campus,None,None,None,None,None,None,None,4300730.0,4.157360e+09,Far Rockaway-Bayswater ...,None,None,None,None,None,8-21 Bay 25 Street,Far Rockaway,NY,11691,,Frederick Douglass Academy (FDA) VI High Schoo...,"Advisory, Graphic Arts Design, Teaching Intern...",Spanish,"Calculus AB, English Language and Composition,...","Biology, Physics B","French, Spanish","Basketball, Cross Country, Indoor Track, Outdo...","Basketball, Cross Country, Indoor Track, Outdo...",,,"Jamaica Hospital Medical Center, Peninsula Hos...","York College, Brooklyn College, St. John's Col...",,"Queens District Attorney, Sports and Arts Foun...","Replications, Inc.",Citibank,New York Road Runners Foundation (NYRRF),"Uniform Required: plain white collared shirt, ...","Extended Day Program, Student Summer Orientati...",This school will provide students with disabil...,ESL,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,8:15 AM,3:00 PM,Priority to New York City residents who attend...,Then to New York City residents,,,,,,,,,None,None,None,http://schools.nyc.gov/schoolportals/21/K559,D to 25th Ave ; N to Ave U ; N to Gravesend - ...,"B1, B3, B4, B6, B64, B82",None,None,260.0,"Film, Music, Talent Show, Holiday Concert, Stu...",,None,None,None,Functionally Accessible,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Lafayette Educational Campus,None,None,None,None,None,None,None,3186454.0,3.068830e+09,Gravesend ...,None,None,None,None,None,2630 Benson Avenue,Brooklyn,NY,11214,,At Life Academy High School for Film and Music...,"College Now, iLEARN courses, Art and Film Prod...",Spanish,,"Biology, English Literature and Composition, E...",,"Basketball, Bowling, Indoor Track, Soccer, Sof...","Basketball

In [35]:
# Code to check school_demographics
query = "SELECT * FROM nyc_schools.school_demographics;"
df = pd.read_sql(query, conn)
df.head(10)



C:\Users\levon\AppData\Local\Temp\ipykernel_11180\774700051.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,grade3,grade4,grade5,grade6,grade7,grade8,grade9,grade10,grade11,grade12,ell_num,ell_percent,sped_num,sped_percent,ctt_num,selfcontained_num,asian_num,asian_per,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,38,52,29,38,,,,,,,36.0,12.8,57,20.3,25.0,9.0,10,3.6,74,26.3,189,67.3,5,1.8,158,56.2,123,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,34,42,46,,,,,,,,38.0,15.6,55,22.6,19.0,15.0,18,7.4,68,28.0,153,63.0,4,1.6,140,57.6,103,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,38,47,40,,,,,,,,52.0,19.9,60,23.0,20.0,14.0,16,6.1,77,29.5,157,60.2,7,2.7,143,54.8,118,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,34,39,49,,,,,,,,48.0,19.0,62,24.6,21.0,17.0,16,6.3,75,29.8,149,59.1,7,2.8,149,59.1,103,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,30,24,38,,,,,,,,40.0,19.2,46,22.1,14.0,14.0,16,7.7,67,32.2,118,56.7,6,2.9,124,59.6,84,40.4
5,01M015,P.S. 015 ROBERTO CLEMENTE,20102011,,96.5,203,13,37,35,33,30,30,25,,,,,,,,30.0,14.8,46,22.7,21.0,9.0,13,6.4,75,36.9,110,54.2,4,2.0,113,55.7,90,44.3
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,,89.4,189,13,31,35,28,25,28,29,,,,,,,,20.0,10.6,40,21.2,23.0,7.0,12,6.3,63,33.3,109,57.7,4,2.1,97,51.3,92,48.7
7,01M019,P.S. 019 ASHER LEVY,20052006,61.5,NaN,402,15,43,55,53,68,59,64,45,,,,,,,37.0,9.2,93,23.1,7.0,37.0,40,10.0,103,25.6,207,51.5,39,9.7,214,53.2,188,46.8
8,01M019,P.S. 019 ASHER LEVY,20062007,61.5,NaN,312,13,37,45,52,47,61,57,,,,,,,,30.0,9.6,72,23.1,13.0,22.0,30,9.6,70,22.4,172,55.1,19,6.1,157,50.3,155,49.7
9,01M019,P.S. 019 ASHER LEVY,20072008,61.5,NaN,338,28,48,46,47,53,48,68,,,,,,,,40.0,11.8,75,22.2,12.0,19.0,42,12.4,72,21.3,186,55.0,22,6.5,162,47.9,176,52.1


In [36]:
# Code to check school_safety_report
query = "SELECT * FROM nyc_schools.school_safety_report;"
df = pd.read_sql(query, conn)
df.head()


C:\Users\levon\AppData\Local\Temp\ipykernel_11180\425001233.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,school_year,building_code,dbn,location_name,location_code,address,borough,geographical_district_code,register,building_name,num_schools,schools_in_building,major_n,oth_n,nocrim_n,prop_n,vio_n,engroupa,rangea,avgofmajor_n,avgofoth_n,avgofnocrim_n,avgofprop_n,avgofvio_n,borough_name,postcode,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,_schools
0,2015-16,K006,17K006,P.S. 006,K006,43 SNYDER AVENUE,K,17.0,712,,None,P.S. 006,0.0,1.0,2.0,1.0,0.0,4C,501-750,0.28,1.11,1.74,0.75,0.53,BROOKLYN,11226.0,40.648590,-73.957138,14.0,40.0,794.0,3117230.0,3.051030e+09,Erasmus ...,1
1,2015-16,K005,16K005,P.S. 005 Dr. Ronald Mcnair,K005,820 HANCOCK STREET,K,16.0,231,,None,P.S. 005 Dr. Ronald Mcnair,0.0,0.0,2.0,0.0,0.0,2C,1-250,0.24,0.71,1.71,0.51,0.32,BROOKLYN,11233.0,40.685695,-73.922463,3.0,41.0,377.0,3039906.0,3.014900e+09,Stuyvesant Heights ...,1
2,2015-16,K003,13K003,P.S. 003 The Bedford Village,K003,50 JEFFERSON AVENUE,K,13.0,450,,None,P.S. 003 The Bedford Village,0.0,0.0,2.0,0.0,0.0,3C,251-500,0.27,0.90,1.67,0.63,0.44,BROOKLYN,11216.0,40.682663,-73.955338,3.0,36.0,227.0,3057406.0,3.019980e+09,Clinton Hill ...,1
3,2015-16,K002,75K141,P.S. K141,K141,655 PARKSIDE AVENUE,K,17.0,374,655 PARKSIDE AVENUE CONSOLIDATED LOCATION,None,Parkside Preparatory Academy | P.S. K141 |Exp...,NaN,NaN,NaN,NaN,NaN,3C,251-500,NaN,NaN,NaN,NaN,NaN,BROOKLYN,11226.0,40.656183,-73.951583,9.0,40.0,802.0,3378201.0,3.050500e+09,Prospect Lefferts Gardens-Wingate ...,3
4,2015-16,K002,84K704,EXPLORE CHARTER SCHOOL(BS),K704,655 PARKSIDE AVENUE,K,17.0,529,655 PARKSIDE AVENUE CONSOLIDATED LOCATION,None,Parkside Preparatory Academy | P.S. K141 |Exp...,NaN,NaN,NaN,NaN,NaN,4C,501-750,NaN,NaN,NaN,NaN,NaN,BROOKLYN,11226.0,40.656183,-73.951583,9.0,40.0,802.0,3378201.0,3.050500e+09,Prospect Lefferts Gardens-Wingate ...,3


## ✅ Task Queries Below

In [37]:
# Answer to question 1, it was here didn't even touch it (NICE)
# Example: Count schools by borough 
query = """
SELECT borough, COUNT(DISTINCT dbn) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough;
"""
df_result = pd.read_sql(query, conn)
df_result

C:\Users\levon\AppData\Local\Temp\ipykernel_11180\6579708.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,school_count
0,Bronx,118
1,Brooklyn,121
2,Manhattan,106
3,Queens,80
4,Staten Island,10


In [38]:
# Code to check Year span in the dataset
query = "SELECT DISTINCT schoolyear FROM nyc_schools.school_demographics ;"
pd.set_option('display.max_columns', None)
df = pd.read_sql(query, conn)
df.head(20)

C:\Users\levon\AppData\Local\Temp\ipykernel_11180\3381056211.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,schoolyear
0,20072008
1,20102011
2,20062007
3,20092010
4,20112012
5,20052006
6,20082009


In [39]:
# Code for average ELL pec per borough
query = """
SELECT hsd.borough, AVG(sd.ell_percent) ::numeric(5,2) AS avg_ell_percent
FROM nyc_schools.school_demographics AS sd
JOIN nyc_schools.high_school_directory AS hsd
    ON sd.dbn = hsd.dbn
WHERE sd.ell_percent IS NOT NULL
GROUP BY hsd.borough
ORDER BY avg_ell_percent DESC;
"""

df_result = pd.read_sql(query, conn)
df_result


C:\Users\levon\AppData\Local\Temp\ipykernel_11180\2211115623.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Manhattan,7.57


In [40]:
# Code for average ELL pec per borough (But per Year)
query = """
SELECT sd.schoolyear, hsd.borough, AVG(sd.ell_percent) ::numeric(5,2) AS avg_ell_percent
FROM nyc_schools.school_demographics AS sd
JOIN nyc_schools.high_school_directory AS hsd
    ON sd.dbn = hsd.dbn
WHERE sd.ell_percent IS NOT NULL
GROUP BY sd.schoolyear, hsd.borough
ORDER BY avg_ell_percent DESC;
"""

df_result = pd.read_sql(query, conn)
df_result


C:\Users\levon\AppData\Local\Temp\ipykernel_11180\3004023576.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,schoolyear,borough,avg_ell_percent
0,20112012,Manhattan,11.96
1,20102011,Manhattan,10.74
2,20092010,Manhattan,8.16
3,20072008,Manhattan,6.58
4,20082009,Manhattan,5.88
5,20052006,Manhattan,5.69
6,20062007,Manhattan,5.67


In [43]:
# Code for top 3 schools in each borough with the highest average sped_percent
query = """
WITH avg_sped AS (
    SELECT sd.dbn, hsd.school_name, hsd.borough, (sd.sped_percent)::numeric AS sped_percent
    FROM nyc_schools.school_demographics AS sd
    JOIN nyc_schools.high_school_directory AS hsd 
        ON sd.dbn = hsd.dbn
    WHERE sd.sped_percent IS NOT NULL

),

ranked_schools AS (
    SELECT *, DENSE_RANK() OVER (PARTITION BY borough ORDER BY sped_percent DESC) AS rank
    FROM avg_sped
)

SELECT borough, school_name, sped_percent
FROM ranked_schools
WHERE rank <= 3
ORDER BY borough, rank, school_name;
"""

df_top_sped = pd.read_sql(query, conn)
df_top_sped



C:\Users\levon\AppData\Local\Temp\ipykernel_11180\1375955810.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_top_sped = pd.read_sql(query, conn)


,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7


## 🧠 Insights

Write your observations, findings, and answers to the task questions here.

Overall Observation:

I didn't clean the dataset, because if I did it today what would I do tomorrow.

Regarding question 2 : The dbn in the school_demographics table includes only one borough and the  schoolyear column is just integer. I did write to code one for overall average ell percent and one with per year breakdown.
